In [1]:
%matplotlib inline

In [2]:
import model_utils
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torch.optim import lr_scheduler
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable

In [3]:
data_dir = 'data/masked_images/data/trainval/breeds_cat/'
test_data_dir = 'data/masked_images/data/test/breeds_cat/'
normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
train_transforms = transforms.Compose([transforms.Resize(224),
                                        #transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       normalize
                                       ])
test_transforms = transforms.Compose([transforms.Resize(224),
                                        #transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       normalize
                                       ])
trainloader, valloader, classes, labels = model_utils.load_split_train_val(data_dir, train_transforms, test_transforms, normalize, .25)

In [4]:
use_gpu = torch.cuda.is_available()

model_cat = torchvision.models.resnet50(pretrained=True)
num_ftrs = model_cat.fc.in_features

model_cat.fc = nn.Linear(num_ftrs, len(classes))

if use_gpu:
    model_cat = model_cat.cuda()

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_cat.parameters(), lr=0.001)

scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)

In [5]:
model_cat = model_utils.train_model(model=model_cat,
                           criterion=criterion,
                           optimizer=optimizer_ft,
                           scheduler=scheduler,
                           trainloader=trainloader,
                           valloader=valloader,
                           num_epochs=15)

Epoch 0/14
--------------------
train Loss: 2.5621 Acc: 0.0985
val Loss: 2.5641 Acc: 0.0985
--------------------
Epoch 1/14
--------------------
train Loss: 2.5318 Acc: 0.1076
val Loss: 2.3625 Acc: 0.1174
--------------------
Epoch 2/14
--------------------
train Loss: 2.4686 Acc: 0.1349
val Loss: 2.1779 Acc: 0.2096
--------------------
Epoch 3/14
--------------------
train Loss: 2.3797 Acc: 0.1670
val Loss: 1.9810 Acc: 0.4193
--------------------
Epoch 4/14
--------------------
train Loss: 2.1589 Acc: 0.2425
val Loss: 1.6794 Acc: 0.5136
--------------------
Epoch 5/14
--------------------
train Loss: 1.8880 Acc: 0.5017
val Loss: 1.6180 Acc: 0.5577
--------------------
Epoch 6/14
--------------------
train Loss: 1.8372 Acc: 0.5325
val Loss: 1.5541 Acc: 0.6184
--------------------
Epoch 7/14
--------------------
train Loss: 1.7960 Acc: 0.5563
val Loss: 1.5277 Acc: 0.6876
--------------------
Epoch 8/14
--------------------
train Loss: 1.7551 Acc: 0.5828
val Loss: 1.5317 Acc: 0.5828
----

In [6]:
saving_path = './trained_model/resnet50_breed_cat.pth'
torch.save(model_cat, saving_path)

In [7]:
model_utils.print_test_acc(model_cat, test_transforms, normalize, test_data_dir)

Overall accuracy for all breeds 53% 

Accuracy of Abyssinian : 50 %
Accuracy of Bengal : 52 %
Accuracy of Birman : 72 %
Accuracy of Bombay : 89 %
Accuracy of British_Shorthair : 65 %
Accuracy of Egyptian_Mau :  0 %
Accuracy of Maine_Coon : 60 %
Accuracy of Persian : 80 %
Accuracy of Ragdoll : 32 %
Accuracy of Russian_Blue :  5 %
Accuracy of Siamese : 62 %
Accuracy of Sphynx : 82 %
